In [1]:
# *Väestöennusteisiin tarvittavien Tilastokeskuskoodien ajo ja haku;
# Haetaan Tilastokeskukselta kuntien välinen muutto - tarvitaan koodissa 3_SEUDUNENN

# 2025 Juuso Heinisuo - Kangasalan kaupunki
# https://github.com/kangasalakehitys/vaestoennuste

from pyjstat import pyjstat
import requests
from functools import reduce
import pandas as pd
import numpy as np
import openpyxl
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Kirjautumistunnukset
un = 'XXXXX'
pw = 'XXXX'

#Exceliin ennustevuodet
file_exc = 'Kuntien_välinen_tulomuutto_Ve1.xlsx'
ENNUSTEVUODET = ["2025","2026","2027","2028","2029","2030","2031","2032","2033","2034","2035"]
ALUE = ["1","1","1","1","1","1","1","1","1","1","1"]
TYYPPI = ["1","1","1","1","1","1","1","1","1","1","1"]

# Tilastokeskuksen kyselyyn parametrit
OMA_KUNTA = ["211"]
IKÄRYHMÄT = ["SSS","0-4","5-9","10-14","15-19","20-24","25-29","30-34","35-39","40-44","45-49","50-54","55-59","60-64","65-"]
VUODET = ["2020","2021","2022","2023","2024"]

SUKUPUOLI = ["SSS","1","2"]
#OSA_ALUEET =  ["211","2111","211101","211101001","211101002","211101003","211102","211102001","211102002","211102003","211103","211103001","211103002","211104","211104001","211104002","211104003","211105","211105001","211105002","211105003","211105004","211106","211106001","211106002","211106003","211106004","211106005","2112","211201","211201001","211201002","211201003","211202","211202001","211202002","211202003","211999999"]
OSA_ALUEET =  ["211"]

# Valitaan oma maakunta
OMA_MAAKUNTA = '06'

# Rajapinnan osoite 12pj -- Kuntaan muuttaneet iän, sukupuolen ja kielen mukaan kunnan osa-alueittain, 2010-2024
POST_URL = 'https://pxhopea2.stat.fi/PXWeb/api/v1/fi/Muuttoliike_ja_vaestonmuutokset/Muuttoliike_ja_vaestonmuutokset_aikasarja/Muuttoliike/muvamuu_12pj.px'
headers = {'Content-Type': 'application/json', 'un': un, 'pw': pw }

In [3]:
query = {
  "query": [
    {
      "code": "Osa-alue",
      "selection": {
        "filter": "item",
        "values": OSA_ALUEET
      }
    },
    {
      "code": "Ikä",
      "selection": {
        "filter": "item",
        "values": IKÄRYHMÄT
      }
    },
    {
      "code": "Kieli",
      "selection": {
        "filter": "item",
        "values": ["SSS"]
      }
    },
    {
      "code": "Sukupuoli",
      "selection": {
        "filter": "item",
        "values": SUKUPUOLI
      }
    },
    {
      "code": "Vuosi",
      "selection": {
        "filter": "item",
        "values": VUODET
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}
resultat = requests.post(POST_URL, headers=headers, json = query)
print(resultat)


<Response [200]>


In [4]:
dataset = pyjstat.Dataset.read(resultat.text)

In [5]:
#Write dataframe
df = dataset.write('dataframe')
#Get all values
df_all = df[(df['Ikä'].str.contains('Yhteensä')) & (df['Sukupuoli'].str.contains('Yhteensä'))]
#Get men
df_men = df[(df['Ikä'].apply(lambda x: 'Yhteensä' not in x)) & (df['Sukupuoli'].str.contains('Miehet'))]
#Get women
df_women = df[(df['Ikä'].apply(lambda x: 'Yhteensä' not in x)) & (df['Sukupuoli'].str.contains('Naiset'))]

df.head(200)


,Osa-alue,Ikä,Kieli,Sukupuoli,Vuosi,Tiedot,value
0,211 Kangasala,Yhteensä,Yhteensä,Yhteensä,2020,Kuntien välinen tulomuutto,2101
1,211 Kangasala,Yhteensä,Yhteensä,Yhteensä,2021,Kuntien välinen tulomuutto,2190
2,211 Kangasala,Yhteensä,Yhteensä,Yhteensä,2022,Kuntien välinen tulomuutto,2212
3,211 Kangasala,Yhteensä,Yhteensä,Yhteensä,2023,Kuntien välinen tulomuutto,2187
4,211 Kangasala,Yhteensä,Yhteensä,Yhteensä,2024,Kuntien välinen tulomuutto,2327
...,...,...,...,...,...,...,...
195,211 Kangasala,60 - 64,Yhteensä,Yhteensä,2020,Kuntien välinen tulomuutto,71
196,211 Kangasala,60 - 64,Yhteensä,Yhteensä,2021,Kuntien välinen tulomuutto,87
197,211 Kangasala,60 - 64,Yhteensä,Yhteensä,2022,Kuntien välinen tulomuutto,62
198,211 Kangasala,60 - 64,Yhteensä,Yhteensä,2023,Kuntien välinen tulomuutto,56


In [6]:
#Year average
df_group_all = round(df_all.groupby(['Osa-alue']).agg(yht=('value', 'mean')),0)
#Year average men
df_group_men = pd.DataFrame(df_men.groupby(['Osa-alue', 'Ikä']).agg(m=('value', 'mean')))
#Year average women
df_group_women = pd.DataFrame(df_women.groupby(['Osa-alue', 'Ikä']).agg(n=('value', 'mean')))

# Convert groupby object to DataFrame 
df_group_all_1 = df_group_all.reset_index()
df_group_men_1 = df_group_men.reset_index()
df_group_women_1 = df_group_women.reset_index()

df_group_women_1.drop(columns=["Ikä"], inplace=True, errors='ignore')

# Repeat value
df_group_all_1 = pd.DataFrame(np.repeat(df_group_all_1.values, repeats=len(df_group_men_1.index), axis=0), columns=df_group_all_1.columns)

df_group = pd.concat([df_group_men_1,df_group_women_1,df_group_all_1], axis=1)
columns_to_drop = ["Osa-alue"]
df_group.drop(columns=columns_to_drop, inplace=True, errors='ignore')

#Sukupuolien osuudet muutoissa
df_group['m_pro'] = (df_group['m'] / df_group['yht'])
df_group['n_pro'] = (df_group['n'] / df_group['yht'])

#Jaetaan 5, jotta saadaan 1-vuodelle luvut
df_group['m_pro'] = (df_group['m_pro'] / 5)
df_group['n_pro'] = (df_group['n_pro'] / 5)

print(df_group)


        Ikä      m      n     yht     m_pro     n_pro
0     0 - 4   86.0   79.8  2203.0  0.007808  0.007245
1   10 - 14   23.6   27.4  2203.0  0.002143  0.002488
2   15 - 19   65.4   98.6  2203.0  0.005937  0.008951
3   20 - 24  155.6  178.4  2203.0  0.014126  0.016196
4   25 - 29  165.8  191.4  2203.0  0.015052  0.017376
5   30 - 34  152.2  151.8  2203.0  0.013818  0.013781
6   35 - 39  103.8   81.8  2203.0  0.009424  0.007426
7   40 - 44   69.2   49.4  2203.0  0.006282  0.004485
8   45 - 49   42.8   41.0  2203.0  0.003886  0.003722
9     5 - 9   41.4   44.6  2203.0  0.003759  0.004049
10  50 - 54   40.6   43.8  2203.0  0.003686  0.003976
11  55 - 59   32.2   45.8  2203.0  0.002923  0.004158
12  60 - 64   33.4   37.4  2203.0  0.003032  0.003395
13     65 -   46.8   73.4  2203.0  0.004249  0.006664


In [7]:
#Generate years
ikä = [x for x in range(100)]
df_ikä = pd.DataFrame(ikä, columns=['Ikä']).apply(pd.to_numeric)

conditions = [
    (df_ikä['Ikä'] <= 4),
    (df_ikä['Ikä'] > 4) & (df_ikä['Ikä'] <= 9),
    (df_ikä['Ikä'] > 9) & (df_ikä['Ikä'] <= 14),
    (df_ikä['Ikä'] > 14) & (df_ikä['Ikä'] <= 19),
    (df_ikä['Ikä'] > 19) & (df_ikä['Ikä'] <= 24),
    (df_ikä['Ikä'] > 24) & (df_ikä['Ikä'] <= 29),
    (df_ikä['Ikä'] > 29) & (df_ikä['Ikä'] <= 34),
    (df_ikä['Ikä'] > 34) & (df_ikä['Ikä'] <= 39),
    (df_ikä['Ikä'] > 39) & (df_ikä['Ikä'] <= 44),
    (df_ikä['Ikä'] > 44) & (df_ikä['Ikä'] <= 49),
    (df_ikä['Ikä'] > 49) & (df_ikä['Ikä'] <= 54),
    (df_ikä['Ikä'] > 54) & (df_ikä['Ikä'] <= 59),
    (df_ikä['Ikä'] > 59) & (df_ikä['Ikä'] <= 64),
    (df_ikä['Ikä'] > 64)
]

# create a list of the values we want to assign for each condition
values = ['0 - 4', '5 - 9', '10 - 14', '15 - 19', '20 - 24', '25 - 29', '30 - 34', '35 - 39', '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64', '65 -']

df_ikä['Ikä_lk'] = np.select(conditions, values, default='Other')

# Merging on multiple columns
res = pd.merge(df_ikä, df_group, left_on=['Ikä_lk'], right_on=['Ikä'], how='inner')

# *Generoidaan sarakenimet;
men = pd.DataFrame(['mst' + str(x) for x in range(100)])
women = pd.DataFrame(['nst' + str(x) for x in range(100)])

res['men'] = men
res['women'] = women



print(res)



    Ikä_x Ikä_lk  Ikä_y     m     n     yht     m_pro     n_pro    men  women
0       0  0 - 4  0 - 4  86.0  79.8  2203.0  0.007808  0.007245   mst0   nst0
1       1  0 - 4  0 - 4  86.0  79.8  2203.0  0.007808  0.007245   mst1   nst1
2       2  0 - 4  0 - 4  86.0  79.8  2203.0  0.007808  0.007245   mst2   nst2
3       3  0 - 4  0 - 4  86.0  79.8  2203.0  0.007808  0.007245   mst3   nst3
4       4  0 - 4  0 - 4  86.0  79.8  2203.0  0.007808  0.007245   mst4   nst4
..    ...    ...    ...   ...   ...     ...       ...       ...    ...    ...
95     95   65 -   65 -  46.8  73.4  2203.0  0.004249  0.006664  mst95  nst95
96     96   65 -   65 -  46.8  73.4  2203.0  0.004249  0.006664  mst96  nst96
97     97   65 -   65 -  46.8  73.4  2203.0  0.004249  0.006664  mst97  nst97
98     98   65 -   65 -  46.8  73.4  2203.0  0.004249  0.006664  mst98  nst98
99     99   65 -   65 -  46.8  73.4  2203.0  0.004249  0.006664  mst99  nst99

[100 rows x 10 columns]


In [8]:
#Create final Excelformat
df_out_men = res[["m_pro", "men"]].copy()
df_out_men = df_out_men.set_index('men').transpose()

df_out_women = res[["n_pro", "women"]].copy()
df_out_women = df_out_women.set_index('women').transpose()

df_out_all = res[["yht"]].copy()
df_out_all_first = df_out_all.iloc[:1]
df_out_all_first.rename(columns={'yht': 'smtyht'}, inplace=True)

list_of_years = list(zip(ENNUSTEVUODET))
list_of_years

df_vuodet = pd.DataFrame(list_of_years,
                  columns=['vuosi'])

df_out_tyyppi = pd.DataFrame(np.array([['1']]))
df_out_tyyppi.columns = ['tyyppi']
df_vuodet.columns = ['vuosi']
df_out_alue = pd.DataFrame(np.array([['1']]))
df_out_alue.columns = ['alue']



final = pd.concat([
    df_out_alue.reset_index(drop=True),
    df_out_tyyppi.reset_index(drop=True),
    df_out_all_first.reset_index(drop=True),
    df_out_men.reset_index(drop=True),
    df_out_women.reset_index(drop=True)], axis=1)

# Repeat value
final2 = pd.DataFrame(np.repeat(final.values, repeats=len(df_vuodet), axis=0), columns=final.columns)

final3 = pd.concat([df_vuodet, final2], axis=1)

print(final3)

final3.to_excel(file_exc, sheet_name='Sheet_name_1', index=False)



   vuosi alue tyyppi  smtyht      mst0      mst1      mst2      mst3  \
0   2025    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   
1   2026    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   
2   2027    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   
3   2028    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   
4   2029    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   
5   2030    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   
6   2031    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   
7   2032    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   
8   2033    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   
9   2034    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   
10  2035    1      1  2203.0  0.007808  0.007808  0.007808  0.007808   

        mst4      mst5  ...     nst90     nst91     nst92     nst93     nst94  \
0   0.007808  0.003759  ...  0.006664  0.006664  0.006